# Mount drive & Import packages

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [ ]:
%cd '/content/gdrive/My Drive/LDS0_K273_DamThiMinhPhuong/Project/Project3_SentimentAnalysis'

/content/gdrive/My Drive/LDS0_K273_DamThiMinhPhuong/Project/Project3_SentimentAnalysis


In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.semi_supervised import LabelSpreading
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split  
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import ExtraTreesClassifier
from imblearn.over_sampling import SMOTE

# Load & Prepare data

In [ ]:
# Load data
df = pd.read_csv('data/foodyClean.csv', index_col=0)
df.shape

(39925, 6)

In [ ]:
df.head()

,restaurant,review_text,review_score,review_score_bins,label,review_text_clean
0,Good Morning Restaurant,Pizza phong cách Ý. Ngán hải sản nên cả nhà gh...,8.2,"(8, 9]",1,rẻ đẹp
1,Hạt Dẻ Trùng Khánh,- Vị trí: chỉ là 1 chiếc xe nho nhỏ nằm ở khu ...,8.0,"(7, 8]",1,đông đối_diện thích hẳn chuyên_nghiệp lắm bự h...
2,Bột - Healthy & Weight Loss Food - Shop Online,Nay lang thang trên BM thấy đang khuyến mãi xo...,8.2,"(8, 9]",1,khuyến_mãi đồ_ăn đa_số khô
3,Bánh Giò Chợ Nguyễn Công Trứ,"Quán nằm trong chợ Nguyễn Công Trứ, cũng dễ tì...",8.2,"(8, 9]",1,nguyễn_công_trứ thẳng đông không_có lộn ngải_c...
4,Quán Quảng - Bún Quảng,Ngãi heo với heo thiệttttt 🐷🐷🐷\n🤤🤤 bún nem chả...,9.0,"(8, 9]",1,ngãi thơm kế sò ốc mềm không_thích dầu_mỡ khôn...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39925 entries, 0 to 39924
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   restaurant         39925 non-null  object 
 1   review_text        39925 non-null  object 
 2   review_score       39925 non-null  float64
 3   review_score_bins  39925 non-null  object 
 4   label              39925 non-null  int64  
 5   review_text_clean  38510 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 2.1+ MB


In [ ]:
df = df.dropna(how='any', axis=0)
df.shape

(38510, 6)

In [ ]:
df = df.reset_index(drop=True)

Lấy ra 400 records để label manually

In [ ]:
# manual_labeling = np.random.randint(0,df.shape[0],size=400) 

In [ ]:
# manual_labeling_data = df.iloc[manual_labeling]

In [ ]:
# manual_labeling_data.to_csv('data/manual_labeling_data.csv', encoding='utf-16', sep='\t', header=True, index=True)

Load 400 records đã label

In [ ]:
manual_labeled_data = pd.read_csv('data/manual_labeling_data.csv', on_bad_lines='skip',
                                  encoding='utf-16', sep='\t', index_col=0)
manual_labeled_data.head()

,restaurant,review_text,review_score,review_score_bins,label,manual_label,review_text_clean
15865,Lotteria - Nguyễn Thái Học,Cách phục vụ và thái độ của nhân viên Lotteria...,3.4,"(3, 4]",0,0,phục_vụ lạt tức_giận đối_xử đừng đối_xử không_...
33385,Kool Tea - Trà Sữa & Ăn Vặt,Trà sữa uống 2 ngăn như vầy mới đã nè. Trà sữa...,8.0,"(7, 8]",1,1,ngăn mát lạnh lắm chiên rẻ
26424,Vintage Coffee House,Quán nằm ngay vị trí trung tâm rất thuận đường...,7.3,"(7, 8]",1,1,thuận đẹp rẻ ổn
36646,Ngon 15 - Lẩu Cá Đuối & Bánh Khọt,"Quán ngon, sạch sẽ, giá cả hợp lý, đặc biệt ph...",9.2,"(9, 10]",1,1,cực đáo dễ_thương mát nổi_tiếng thích
17022,Mì Quảng Ếch Bếp Trang - Ông Ích Khiêm,Mi order ve ks cach quan hon 12km. Giao den th...,8.2,"(8, 9]",1,1,không pack xop


In [ ]:
manual_labeled_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 400 entries, 15865 to 15732
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   restaurant         400 non-null    object 
 1   review_text        400 non-null    object 
 2   review_score       400 non-null    float64
 3   review_score_bins  400 non-null    object 
 4   label              400 non-null    int64  
 5   manual_label       400 non-null    int64  
 6   review_text_clean  400 non-null    object 
dtypes: float64(1), int64(2), object(4)
memory usage: 25.0+ KB


In [ ]:
manual_labeled_data.label.value_counts()

1    326
0     74
Name: label, dtype: int64

In [ ]:
manual_labeled_data.manual_label.value_counts()

1    319
0     81
Name: manual_label, dtype: int64

Ta nhận thấy không có nhiều khác biệt giữa label theo review_score và manual label

In [ ]:
# Lọc ra unlabeled data còn lại
unlabeled_df = df.loc[~ df.index.isin(manual_labeled_data.index)]
unlabeled_df.shape

(38110, 6)

In [ ]:
unlabeled_df.head()

,restaurant,review_text,review_score,review_score_bins,label,review_text_clean
0,Good Morning Restaurant,Pizza phong cách Ý. Ngán hải sản nên cả nhà gh...,8.2,"(8, 9]",1,rẻ đẹp
1,Hạt Dẻ Trùng Khánh,- Vị trí: chỉ là 1 chiếc xe nho nhỏ nằm ở khu ...,8.0,"(7, 8]",1,đông đối_diện thích hẳn chuyên_nghiệp lắm bự h...
2,Bột - Healthy & Weight Loss Food - Shop Online,Nay lang thang trên BM thấy đang khuyến mãi xo...,8.2,"(8, 9]",1,khuyến_mãi đồ_ăn đa_số khô
3,Bánh Giò Chợ Nguyễn Công Trứ,"Quán nằm trong chợ Nguyễn Công Trứ, cũng dễ tì...",8.2,"(8, 9]",1,nguyễn_công_trứ thẳng đông không_có lộn ngải_c...
4,Quán Quảng - Bún Quảng,Ngãi heo với heo thiệttttt 🐷🐷🐷\n🤤🤤 bún nem chả...,9.0,"(8, 9]",1,ngãi thơm kế sò ốc mềm không_thích dầu_mỡ khôn...


In [ ]:
tfidf = TfidfVectorizer(max_features=8000) 
all_features = tfidf.fit_transform(df['review_text_clean'])
all_features = all_features.toarray()

In [ ]:
all_features.shape

(38510, 8000)

In [ ]:
# Train data (~99%)
features_1 = all_features[unlabeled_df.index]
target_1 = df.loc[unlabeled_df.index]['label']

In [ ]:
# Validation data (400 records ~ 1%)
X_test_ml = all_features[manual_labeled_data.index]
y_test_ml = manual_labeled_data['manual_label']

In [ ]:
# Seperate list for Unlabeled Samples
Unlabeled = np.copy(target_1)
# Select a mask of 20% of the train data
y_mask = np.random.rand(len(target_1)) <=0.2
# Set the non-masked subset to be unlabeled
Unlabeled[~y_mask] = -1

In [ ]:
# Inspect
Unlabeled.T

array([-1, -1, -1, ..., -1, -1, -1])

In [ ]:
print("Unlabeled samples in training set:", sum(1 for x in Unlabeled if x == -1))

Unlabeled samples in training set: 30401


# Build Model

## Label Spreading - using manual label

In [ ]:
# Label Spreading
label_prop_model = LabelSpreading()

# Fit to Label Spreading 
label_prop_model.fit(features_1, Unlabeled) #input, output

# Predict the Labels for Unlabeled Samples
pred_ls = label_prop_model.predict(features_1)

# #Accuracy of Prediction
print('Accuracy of Label Spreading', label_prop_model.score(features_1, pred_ls))   #score tính dựa trên xác suất

Accuracy of Label Spreading 1.0


In [ ]:
pred_ls.T

array([1, 1, 1, ..., 1, 1, 0])

In [ ]:
len(pred_ls)

38110

In [ ]:
print('Unlabeled samples in training set', sum(1 for x in pred_ls if x==-1))

Unlabeled samples in training set 0


In [ ]:
# Define Model label_spread
model_ls = BernoulliNB()

# Fit 90% dữ liệu, trong đó 20% có nhãn, còn lại là label spreading
model_ls.fit(features_1, pred_ls)

#Prediction (10%)
yls_pred = model_ls.predict(X_test_ml)

#Accuracy Score
acc_ls = accuracy_score(y_test_ml, yls_pred)
print('Accuracy:','{:.2%}'.format(acc_ls))

Accuracy: 84.25%


In [ ]:
print(confusion_matrix(y_test_ml, yls_pred))

[[ 23  58]
 [  5 314]]


In [ ]:
print(classification_report(y_test_ml, yls_pred))

              precision    recall  f1-score   support

           0       0.82      0.28      0.42        81
           1       0.84      0.98      0.91       319

    accuracy                           0.84       400
   macro avg       0.83      0.63      0.67       400
weighted avg       0.84      0.84      0.81       400



## Label Spreading - using label by 'review_score'

In [ ]:
target_number = df['label']

In [ ]:
# 90% train (áp dụng LabelSpreading, self-training => train ) / 10% validation (test)
features_1, X_test_st, target_1, y_test_st = train_test_split(all_features, target_number, test_size=0.1)

In [ ]:
# Seperate list for Unlabeled Samples
Unlabeled = np.copy(target_1)
# Select a mask of 20% of the train dataset
y_mask = np.random.rand(len(target_1)) <=0.2
# Set the non-masked subset to be unlabeled
Unlabeled[~y_mask] = -1

In [ ]:
# Inspect
Unlabeled.T

array([ 1, -1, -1, ..., -1, -1, -1])

In [ ]:
print("Unlabeled samples in training set:", sum(1 for x in Unlabeled if x == -1))

Unlabeled samples in training set: 27690


In [ ]:
# Label Spreading
label_prop_model = LabelSpreading()

# Fit to Label Spreading 
label_prop_model.fit(features_1, Unlabeled) #input, output

# Predict the Labels for Unlabeled Samples
pred_lb = label_prop_model.predict(features_1)

# #Accuracy of Prediction
print('Accuracy of Label Spreading', label_prop_model.score(features_1, pred_lb))   #score tính dựa trên xác suất

Accuracy of Label Spreading 1.0


In [ ]:
print('Unlabeled samples in training set', sum(1 for x in pred_lb if x==-1))

Unlabeled samples in training set 0


In [ ]:
# Define Model
model_lb = BernoulliNB()

# Fit 90% dữ liệu, trong đó 20% có nhãn, còn lại là label spreading
model_lb.fit(features_1, pred_lb)

#Prediction (10%)
yl_pred = model_lb.predict(X_test_st)

#Accuracy Score
acc_ls = accuracy_score(y_test_st, yl_pred)
print('Accuracy:','{:.2%}'.format(acc_ls))

Accuracy: 87.51%


In [ ]:
print(confusion_matrix(y_test_st, yl_pred))
print(classification_report(y_test_ml, yls_pred))

[[ 253  395]
 [  86 3117]]
              precision    recall  f1-score   support

           0       0.82      0.28      0.42        81
           1       0.84      0.98      0.91       319

    accuracy                           0.84       400
   macro avg       0.83      0.63      0.67       400
weighted avg       0.84      0.84      0.81       400



In [ ]:
algorithms = [LogisticRegression(), NearestCentroid(), ExtraTreesClassifier()]
for algo in algorithms:
    # Define Model
    model = algo
    # Fit 90% dữ liệu, trong đó 20% có nhãn, còn lại là label spreading
    model.fit(features_1, pred_lb)
    #Prediction (10%)
    yl_pred = model.predict(X_test_st)
    #Accuracy Score
    acc_ls = accuracy_score(y_test_st, yl_pred)
    print('Accuracy:',algo.__class__.__name__,'{:.2%}'.format(acc_ls))

Accuracy: LogisticRegression 87.12%
Accuracy: NearestCentroid 83.33%
Accuracy: ExtraTreesClassifier 86.24%


## Self-training - using label by 'review_score'

In [ ]:
# SelfTrainingClassifier
self_training_model = SelfTrainingClassifier(BernoulliNB(), 
                                             threshold=0.95) 

In [ ]:
# Fit
self_training_model.fit(features_1, Unlabeled)  #90%: 20% có nhãn, 80% ko có
# Predict the Labels for Unlabeled Samples
pred_st = self_training_model.predict(features_1)

In [ ]:
# #Accuracy of Prediction
print('Accuracy of Self-training:', self_training_model.score(features_1, pred_st))

Accuracy of Self-training: 1.0


In [ ]:
pred_st.T

array([1, 1, 1, ..., 1, 0, 1])

In [ ]:
print("Unlabeled samples in training set:", sum(1 for x in pred_st if x == -1))

Unlabeled samples in training set: 0


In [ ]:
# Define Model
models = BernoulliNB()
#Fit
models.fit(features_1, pred_st)

# Prediction
ys_pred = models.predict(X_test_st)

# Accuracy Score
acc_st = accuracy_score(y_test_st, ys_pred)

print('BernoulliNB Model Accuracy (after Self-training):', acc_st)

BernoulliNB Model Accuracy (after Self-training): 0.8327707089067775


In [ ]:
print(confusion_matrix(y_test_st, ys_pred))

[[  44  604]
 [  40 3163]]


In [ ]:
print(classification_report(y_test_st, ys_pred))

              precision    recall  f1-score   support

           0       0.52      0.07      0.12       648
           1       0.84      0.99      0.91      3203

    accuracy                           0.83      3851
   macro avg       0.68      0.53      0.51      3851
weighted avg       0.79      0.83      0.78      3851



Nhận xét:
- Accuracy của các model đều khá tốt 83 - 87% tuy nhiên F1 score trên class 0 lại rất thấp. => Phương pháp này không phù hợp với dataset

# Kết luận:
- Sau khi thực hiện rất nhiều phương pháp học: Supervised Learning, Semi-supervised Learning, RNN, ta thấy rằng model xây dựng dựa trên thuật toán SGDClassifier trên imbalanced data cho kết quả tốt nhất
## => Đề xuất: model SGDClassifier